## Notebook for face verification

In [0]:
!kill -9 -1

In [2]:
import tensorflow as tf
import tensorflow.keras as K
import os
import cv2
import PIL.Image
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

print(tf.__version__)
print(K.__version__)

2.2.0
2.3.0-tf


In [0]:
#Connect with drive

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# read data from drive
with open('drive/My Drive/Face Verification/haarcascade_frontalface_default.xml', 'r') as f:
    haar_file = f.read()

print(haar_file)
#with open('drive/My Drive/Smart Reply/10000_labels.csv', 'r') as f:
    #labels = f.read()

# Take 20 photos

In [4]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# All the faces data will be present this folder 
dataset = './dataset'

if not os.path.exists(dataset):
    os.mkdir(dataset)
sub_data = input("Enter your username: ")
#sub_data_gray = input("Enter your username for gray scale: ")

# Use the username as path name
path = os.path.join(dataset, sub_data) 
path_gray = 'dataset/{}{}'.format(sub_data,'_gray')

# Add a verfication for this step
if not os.path.exists(path):
    os.mkdir(path)
    os.mkdir(path_gray)    

def take_photo(filename='dataset/name/photo.jpg', quality=1):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

Enter your username: george


In [6]:
from IPython.display import Image

# Take 20 photos
for i in range(20):
  try:
    filename = take_photo(filename='dataset/{}/photo{}.jpg'.format(sub_data,i))
    print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
    #display(Image(filename))
  except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
    print(str(err))



<IPython.core.display.Javascript object>

Saved to dataset/george/photo0.jpg


<IPython.core.display.Javascript object>

Saved to dataset/george/photo1.jpg


<IPython.core.display.Javascript object>

...

Saved to dataset/george/photo17.jpg


<IPython.core.display.Javascript object>

Saved to dataset/george/photo18.jpg


<IPython.core.display.Javascript object>

Saved to dataset/george/photo19.jpg


In [0]:
# zip saved model folder
!zip -r /content/dataset/george.zip /content/dataset/george

## If you come later unzip the file with photos

In [0]:
if not os.path.exists('dataset'):
    os.mkdir('dataset')

if not os.path.exists('dataset_gray'):
    os.mkdir('dataset_gray')

!unzip george.zip -d dataset
!unzip Kim.zip -d dataset
!unzip Bean.zip -d dataset

In [0]:

path_gray_george = 'dataset_gray/{}'.format('george')

# Add a verfication for this step
if not os.path.exists(path_gray_george):
    os.mkdir(path_gray_george) 

sub_data = 'george'

def create_dataset_george():
    # The file containing the pretrained classifier 
    haar_file = 'drive/My Drive/Face Verification/haarcascade_frontalface_default.xml'
        
    # Image to be resized to this shape
    (width, height) = (180, 200)     
    
    # Make the cascade classifier object
    face_cascade = cv2.CascadeClassifier(haar_file) 
    #webcam = cv2.VideoCapture(0)  

    # The program loops until it has 20 images of the face. 
    count = 0
    while count < 20:
        # Read from file
        # Creates blueish file
        im = PIL.Image.open('dataset/{}/photo{}.jpg'.format(sub_data,count))
        # Creates images with normal color
        #im = cv2.imread('dataset/{}/photo{}.jpg'.format(sub_data,count))
        im = np.array(im)
        #print(im.shape)
        
        # Convert to grayscale
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) 
        # Detect the face
        faces = face_cascade.detectMultiScale(gray, 1.3, 4) 
        
        face_resize = None
        for (x, y, w, h) in faces:
            # The classifier seemed to scrap the chin and hair. Adjustments made to accomodate those.
            face = im[y-60 : y+h+60, x-20 : x+w+20] 
            face_resize = cv2.resize(face, (width, height)) 
            cv2.imwrite('% s/% s.png' % (path_gray_george, 'photoGeorge{}'.format(count)), face_resize) 
        count += 1

        #cv2.imshow('OpenCV', im) 
        #key = cv2.waitKey(100) 
        #if key == 27: 
            #break

# Call this function whenever you need to create a dataset of the person's images
create_dataset_george()

In [9]:

path_gray_Bean = 'dataset_gray/{}'.format('Bean')

# Add a verfication for this step
if not os.path.exists(path_gray_Bean):
    os.mkdir(path_gray_Bean) 

sub_data = 'Bean'
dir_contents_Bean = os.listdir('dataset/Bean')
print(len(dir_contents_Bean))

def create_dataset_Bean():
   
    # Image to be resized to this shape
    (width, height) = (180, 200) 

    # The program loops until it has 20 images of the face. 
    count = 0
    while count < 20:
        # Read from file
        # Creates blueish file
        im = PIL.Image.open('dataset/Bean/{}'.format(dir_contents_Bean[count]))
        print('dataset/Bean/{}'.format(dir_contents_Bean[count]))
        # Creates images with normal color
        #im = cv2.imread('dataset/Bean/{}.jpg'.format(dir_contents_Bean[count]))
        im = np.array(im)
        print(im.shape)
        
        # Convert to grayscale
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) 

        face_resize = cv2.resize(gray, (width, height))
        print(face_resize.shape)

        cv2.imwrite('% s/% s.png' % (path_gray_Bean, 'photoBean{}'.format(count)), face_resize) 
        #cv2.waitKey()

        count += 1

# Call this function whenever you need to create a dataset of the person's images
create_dataset_Bean()

20
dataset/Bean/014.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/009.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/003.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/002.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/013.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/012.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/006.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/016.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/005.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/011.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/008.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/018.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/020.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/004.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/001.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/015.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/007.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/019.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/017.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/010.jpg
(256, 256, 3)
(200, 180, 3)


In [10]:

path_gray_Kim = 'dataset_gray/{}'.format('Kim')

# Add a verfication for this step
if not os.path.exists(path_gray_Kim):
    os.mkdir(path_gray_Kim) 

sub_data = 'Kim'
dir_contents_Kim = os.listdir('dataset/Kim')
print(len(dir_contents_Kim))

def create_dataset_Kim():  
    # Image to be resized to this shape
    (width, height) = (180, 200)      

    # The program loops until it has 20 images of the face. 
    count = 0
    while count < 20:
        # Read from file
        im = PIL.Image.open('dataset/Kim/{}'.format(dir_contents_Kim[count]))
        print('dataset/Bean/{}'.format(dir_contents_Kim[count]))
        im = np.array(im)
        print(im.shape)
        
        # Convert to grayscale
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) 

        face_resize = cv2.resize(gray, (width, height))
        print(face_resize.shape)

        cv2.imwrite('% s/% s.png' % (path_gray_Kim, 'photoKim{}'.format(count)), face_resize) 

        count += 1

# Call this function whenever you need to create a dataset of the person's images
create_dataset_Kim()

20
dataset/Bean/014.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/009.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/023.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/003.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/002.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/012.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/006.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/016.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/027.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/005.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/011.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/008.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/018.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/020.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/004.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/001.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/021.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/015.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/007.jpg
(256, 256, 3)
(200, 180, 3)
dataset/Bean/017.jpg
(256, 256, 3)
(200, 180, 3)


## Import Keras layers

In [11]:
from tensorflow.keras import backend as K, models
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2
from tensorflow.keras.activations import relu
import os
from os.path import join as join_
import numpy as np
from PIL import Image

Using TensorFlow backend.


In [12]:
# Setting up the dataset

SET_DIR = 'dataset_gray/'
NUM_CLASSES = len(os.listdir('dataset_gray'))
print(NUM_CLASSES)

# The shape which VGG19 accepts as input and thus each image is resized to
image_shape = (224, 224, 3)

# NUM_EXAMPLES is the number of (A,P,N) triplets chosen for the same class (N belongs to a different class of course)
NUM_EXAMPLES = 15

# Triplets list will contain anchor(A), positive(P) and negative(N) triplets.
triplets = []
A = P = N = []
A_str = P_str = N_str = []

# creating anchor, positive, negative triplets
for _ in range(NUM_EXAMPLES):
    for direc in os.listdir(SET_DIR):
        dir_path = SET_DIR + direc
        dir_contents = os.listdir(dir_path)
        length = len(dir_contents)
        print(length)
        
        A_ran = dir_contents[np.random.randint(0, length)]
        A_str = Image.open(join_(dir_path, A_ran))
        anchor = np.asarray(A_str)/255
        # anchor.shape = (200, 180, 3)        
        # Padding with zeros for each channel in RGB
        anchor = np.array([np.pad(a, ((22,22), (12,12)), 'constant') for a in anchor.T]).T
        
        P_ran = dir_contents[np.random.randint(0, length)]
        P_str = Image.open(join_(dir_path, P_ran))
        positive = np.asarray(P_str)/255
        positive = np.array([np.pad(a, ((22,22), (12,12)), 'constant') for a in positive.T]).T
        
        neg_dir = os.listdir(SET_DIR)[np.random.randint(NUM_CLASSES)]
        while neg_dir == direc: 
            neg_dir = os.listdir(SET_DIR)[np.random.randint(NUM_CLASSES)]
            
        length_negative = len(os.listdir(SET_DIR + neg_dir))

        N_ran = os.listdir(SET_DIR + neg_dir)[np.random.randint(0, length_negative)]
        N_str = Image.open(join_(SET_DIR + neg_dir, N_ran))
        negative = np.asarray(N_str)/255
        
        negative = np.array([np.pad(a, ((22,22), (12,12)), 'constant') for a in negative.T]).T
        
        # append triplet
        #triplets.append([anchor, positive, negative])
        #A.append(anchor)
        #P.append(positive)
        #N.append(negative)

        triplets.append([A_ran, P_ran, N_ran])
        A.append(A_ran)
        P.append(P_ran)
        N.append(N_ran)


3
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20


In [16]:
print(triplets[1:100])
#print(A[0])
#print(P)
#print(N)

[['photoKim10.png', 'photoKim7.png', 'photoBean4.png'], ['photoGeorge9.png', 'photoGeorge3.png', 'photoBean3.png'], ['photoBean13.png', 'photoBean15.png', 'photoGeorge10.png'], ['photoKim1.png', 'photoKim11.png', 'photoGeorge0.png'], ['photoGeorge2.png', 'photoGeorge4.png', 'photoBean12.png'], ['photoBean17.png', 'photoBean12.png', 'photoGeorge6.png'], ['photoKim3.png', 'photoKim15.png', 'photoBean1.png'], ['photoGeorge16.png', 'photoGeorge16.png', 'photoBean11.png'], ['photoBean5.png', 'photoBean11.png', 'photoKim8.png'], ['photoKim11.png', 'photoKim0.png', 'photoBean3.png'], ['photoGeorge19.png', 'photoGeorge9.png', 'photoBean4.png'], ['photoBean14.png', 'photoBean11.png', 'photoGeorge2.png'], ['photoKim11.png', 'photoKim2.png', 'photoGeorge6.png'], ['photoGeorge13.png', 'photoGeorge19.png', 'photoKim15.png'], ['photoBean8.png', 'photoBean2.png', 'photoKim8.png'], ['photoKim4.png', 'photoKim11.png', 'photoGeorge4.png'], ['photoGeorge6.png', 'photoGeorge7.png', 'photoBean2.png'], ['ph